<a target="_blank" href="https://colab.research.google.com/github/hike-lab/address-data-guide/blob/main/chapter-3/chapter3-5/3_5_%EC%A3%BC%EC%86%8C_%EB%8D%B0%EC%9D%B4%ED%84%B0_%ED%99%9C%EC%9A%A9%ED%95%98%EA%B8%B0.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 3.5 주소 데이터 활용하기


## 라이브러리 설정

In [ ]:
!pip install mapboxgl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chroma-py: filename=chroma_py-0.1.0.dev1-py3-none-any.whl size=5092 sha256=95452482dba314900c066fff1e563648273f57c3272af99ebbebaebef04f534c
  Stored in directory: /root/.cache/pip/wheels/45/89/c1/b8b2a40648c05fe0c0618b1de25aecde35c972e19cd4cb2341
Successfully built chroma-py


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import plotly.express as px
import glob
import geopandas as gpd
import mapboxgl
from mapboxgl.viz import *
import geojson
from mapboxgl.utils import create_color_stops, create_numeric_stops

## 데이터 불러오기

도로명주소데이터는 [3.3]도로명주소_데이터 EDA에서 사용한 데이터와 동일합니다.    
인구데이터는 행정안전부에서 제공하는 행정동별 주민등록 인구 및 세대현황의 2024년 1월 전체시군구현황 데이터입니다.    
면적데이터는 주소기반산업지원서비스에서 제공하는 제공하는 주소 중 구역의 도형의 2024년 1월 전체자료입니다.

모든 데이터는 아래 구글 드라이브 링크에서 다운받아서 사용하거나, 사이트에서 직접 신청, 다운 받을 수 있습니다.
* [깃헙]()
* [인구데이터](https://jumin.mois.go.kr/)
* [면적데이터](https://business.juso.go.kr/addrlink/elctrnMapProvd/geoDBDwldList.do?menu=%EA%B5%AC%EC%97%AD%EC%9D%98%20%EB%8F%84%ED%98%95) (신청 방식)

In [ ]:
df = pd.read_csv('3-5_total-road-name-address_2401.csv', encoding='utf-8', dtype=str)
df.head()

,도로명주소관리번호,법정동코드,시도명,시군구명,법정읍면동명,법정리명,산여부,지번본번(번지),지번부번(호),도로명코드,...,행정동코드,행정동명,기초구역번호(우편번호),이전도로명주소,효력발생일,공동주택구분,이동사유코드,건축물대장건물명,시군구용건물명,비고
0,27110101300701700008800000,2711010100,대구광역시,중구,동인동1가,NaN,0,2,1,271103007017,...,2711051700,동인동,41911,NaN,20110729,0,NaN,NaN,대구광역시 동인청사,NaN
1,27110101300701700009600000,2711010100,대구광역시,중구,동인동1가,NaN,0,30,1,271103007017,...,2711051700,동인동,41911,NaN,20110729,0,NaN,NaN,NaN,NaN
2,27110101300701700010000000,2711010100,대구광역시,중구,동인동1가,NaN,0,49,3,271103007017,...,2711051700,동인동,41911,NaN,20110729,0,NaN,NaN,NaN,NaN
3,27110101314100500018000000,2711010100,대구광역시,중구,동인동1가,NaN,0,289,2,271103141005,...,2711051700,동인동,41908,NaN,20111028,0,NaN,NaN,동인치안센터,NaN
4,27110101314100500018200000,2711010100,대구광역시,중구,동인동1가,NaN,0,292,1,271103141005,...,2711051700,동인동,41908,NaN,20110729,0,NaN,NaN,대구광역시시청어린이집,NaN


In [ ]:
df_pop = pd.read_excel('3-5_주민등록인구및세대현황_월간.xlsx', header=2, dtype=str)
df_pop

,행정기관코드,행정기관,총인구수,세대수,세대당 인구,남자 인구수,여자 인구수,남여 비율
0,1100000000,서울특별시,"9,384,325","4,471,017",2.10,"4,538,354","4,845,971",0.94
1,1111000000,서울특별시 종로구,"139,378","72,101",1.93,"67,240","72,138",0.93
2,1111051500,서울특별시 종로구 청운효자동,"11,323","5,043",2.25,"5,170","6,153",0.84
3,1111053000,서울특별시 종로구 사직동,"8,992","4,535",1.98,"3,980","5,012",0.79
4,1111054000,서울특별시 종로구 삼청동,"2,250","1,126",2.00,"1,075","1,175",0.91
...,...,...,...,...,...,...,...,...
3907,5013058000,제주특별자치도 서귀포시 서홍동,"11,131","4,603",2.42,"5,461","5,670",0.96
3908,5013059000,제주특별자치도 서귀포시 대륜동,"15,271","6,905",2.21,"7,658","7,613",1.01
3909,5013060000,제주특별자치도 서귀포시 대천동,"13,639","6,487",2.10,"6,937","6,702",1.04
3910,5013061000,제주특별자치도 서귀포시 중문동,"12,409","6,041",2.05,"6,305","6,104",1.03


In [ ]:
# 시도 면적데이터

# 해당 경로에서 '_CTPRVN.shp'로 끝나는 모든 파일 불러오기
sido_file_list = glob.glob('3-4_geojson/*/*_CTPRVN.shp')

sido_geojson = pd.DataFrame()

# 모든 파일 geopandas로 불러오고 하나의 df로 합쳐주기
for file in tqdm(sido_file_list):
    df_tmp = gpd.read_file(file, encoding="cp949")
    sido_geojson = pd.concat([sido_geojson, df_tmp])

sido_geojson.columns = ["code", "eng_nm", "kor_nm", "geometry"]
sido_geojson["code"] = sido_geojson["code"].apply(lambda x: str(x) + "00000000")
sido_geojson.reset_index(inplace=True, drop=True)

print(sido_geojson.shape)
sido_geojson.head()

In [ ]:
# 시군구 면적데이터

# 해당 경로에서 '_SIG.shp'로 끝나는 모든 파일 불러오기
sigungu_file_list = glob.glob('geojson/*/*_SIG.shp')

sigungu_geojson = pd.DataFrame()

# 모든 파일 geopandas로 불러오고 하나의 df로 합쳐주기
for file in tqdm(sigungu_file_list):
    df_tmp = gpd.read_file(file, encoding="cp949")
    sigungu_geojson = pd.concat([sigungu_geojson, df_tmp])

sigungu_geojson.columns = ["code", "eng_nm", "kor_nm", "geometry"]
sigungu_geojson["code"] = sigungu_geojson["code"].apply(lambda x: str(x) + "00000")
sigungu_geojson.reset_index(inplace=True, drop=True)

print(sigungu_geojson.shape)
sigungu_geojson.head()

100%|██████████| 17/17 [00:20<00:00,  1.21s/it]


(252, 4)


,code,eng_nm,kor_nm,geometry
0,4413100000,"Dongnam-gu, Cheonan-si",천안시 동남구,"POLYGON ((981821.389 1876846.713, 981931.111 1..."
1,4413300000,"Seobuk-gu, Cheonan-si",천안시 서북구,"POLYGON ((965794.161 1885898.909, 965821.523 1..."
2,4415000000,Gongju-si,공주시,"POLYGON ((952436.598 1853608.129, 952532.895 1..."
3,4418000000,Boryeong-si,보령시,"MULTIPOLYGON (((862402.125 1805616.130, 862475..."
4,4420000000,Asan-si,아산시,"POLYGON ((961587.843 1882485.024, 961618.275 1..."


In [ ]:
## 시군구 면적데이터에 세종특별자치시는 시군구가 없어서 한 행만 포함되어 있음
sigungu_geojson.loc[sigungu_geojson['code']=='3611000000']

,code,eng_nm,kor_nm,geometry
134,3611000000,Sejong-si,세종특별자치시,"POLYGON ((971235.936 1844386.985, 971234.055 1..."


## 분석1. 인구 대비 도로명주소 개수

In [ ]:
## 시각화 공통 함수

def graph_viz(df, title):
    fig = px.bar(df,
            x = df.index,
            y = df.columns,
            color = df.index,
            title=f"{title}",
            color_discrete_sequence=px.colors.qualitative.Set2,
            height=400
    )

    fig.update_layout(yaxis={'visible': True, 'showticklabels': False},
                      plot_bgcolor="#F9FAFB",  showlegend=False, paper_bgcolor='#F9FAFB', margin=dict(l=0, r=0, t=40),
                      title={'text': "%s" % title,'y':0.95,'x':0,'xanchor': 'left','yanchor': 'top'},)
    fig.update_xaxes(title=None)
    fig.update_yaxes(title=None, gridcolor='lightgray', mirror=True)
    fig.write_html("%s.html" % title)
    fig.show()

### 인구데이터 정제하기

In [ ]:
df_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3912 entries, 0 to 3911
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   행정기관코드  3912 non-null   object
 1   행정기관    3912 non-null   object
 2   총인구수    3912 non-null   object
 3   세대수     3912 non-null   object
 4   세대당 인구  3912 non-null   object
 5   남자 인구수  3912 non-null   object
 6   여자 인구수  3912 non-null   object
 7   남여 비율   3912 non-null   object
dtypes: object(8)
memory usage: 244.6+ KB


In [ ]:
df_pop.isnull().sum()

행정기관코드    0
행정기관      0
총인구수      0
세대수       0
세대당 인구    0
남자 인구수    0
여자 인구수    0
남여 비율     0
dtype: int64

In [ ]:
len(df_pop['행정기관'].unique())

3911

In [ ]:
# 중복 데이터 확인하기
df_pop[df_pop['행정기관'].duplicated(keep=False)]

,행정기관코드,행정기관,총인구수,세대수,세대당 인구,남자 인구수,여자 인구수,남여 비율
1261,3600000000,세종특별자치시,"386,944","160,942",2.40,"192,704","194,240",0.99
1262,3611000000,세종특별자치시,"386,944","160,942",2.40,"192,704","194,240",0.99


In [ ]:
# 36110#####가 올바른 코드이므로, 36000#####를 삭제함
df_pop = df_pop[df_pop['행정기관코드']!=3600000000]
df_pop.reset_index(inplace=True, drop=True)

# 확인하기
df_pop[df_pop['행정기관'].duplicated(keep=False)]

,행정기관코드,행정기관,총인구수,세대수,세대당 인구,남자 인구수,여자 인구수,남여 비율
1261,3600000000,세종특별자치시,"386,944","160,942",2.40,"192,704","194,240",0.99
1262,3611000000,세종특별자치시,"386,944","160,942",2.40,"192,704","194,240",0.99


### 인구 대비 도로명주소 개수 계산하기

#### 시도별

In [ ]:
# 시도별 도로명주소 개수 데이터
sido = pd.DataFrame(df.groupby('시도명')["도로명주소관리번호"].count()) # 데이터프레임 생성하기
sido.reset_index(inplace=True)
sido['시도명'] = sido['시도명'].apply(lambda x:x.strip()) # 좌우 공백 없애기
sido['도로명주소관리번호'] = sido['도로명주소관리번호'].astype(int) # int로 데이터 타입 바꾸기

# 시도별 인구수 데이터
df_pop['행정기관'] = df_pop['행정기관'].apply(lambda x:x.strip())
sido_pop = df_pop[df_pop['행정기관'].isin(list(sido['시도명']))].copy() # 데이터프레임 생성하기
sido_pop['행정기관'] = sido_pop['행정기관'].apply(lambda x:x.strip()) # 좌우 공백 없애기
sido_pop['행정기관코드'] = sido_pop['행정기관코드'].astype('str') # str로 데이터 타입 바꾸기
sido_pop['총인구수'] = sido_pop['총인구수'].str.replace(',', '').astype(int) # int로 데이터 타입 바꾸기

In [ ]:
# 하나의 데이터프레임으로 합치기
sido_address_pop = pd.merge(sido, sido_pop, left_on='시도명', right_on='행정기관')
sido_address_pop.drop('행정기관', axis=1,  inplace=True)
sido_address_pop.rename(columns={'도로명주소관리번호':'도로명개수'}, inplace=True)

# 인구 대비 도로명주소 개수 계산하기
sido_address_pop["인구 대비 도로명주소 개수"] = sido_address_pop.apply(lambda row: row["도로명개수"] / row["총인구수"], axis=1)
sido_address_pop

,시도명,도로명개수,행정기관코드,총인구수,세대수,세대당 인구,남자 인구수,여자 인구수,남여 비율,인구 대비 도로명주소 개수
0,강원특별자치도,364686,5100000000,1526243,"760,361",2.01,"767,496","758,747",1.01,0.238944
1,경기도,1020563,4100000000,13635250,"5,984,576",2.28,"6,857,867","6,777,383",1.01,0.074847
2,경상남도,653480,4800000000,3248703,"1,525,692",2.13,"1,636,010","1,612,693",1.01,0.201151
3,경상북도,713862,4700000000,2551370,"1,281,979",1.99,"1,288,999","1,262,371",1.02,0.279796
4,광주광역시,120474,2900000000,1418241,"655,359",2.16,"700,377","717,864",0.98,0.084946
5,대구광역시,229727,2700000000,2373844,"1,094,529",2.17,"1,166,110","1,207,734",0.97,0.096774
6,대전광역시,114026,3000000000,1441562,"680,459",2.12,"719,008","722,554",1.00,0.079099
7,부산광역시,301822,2600000000,3290964,"1,564,763",2.10,"1,603,859","1,687,105",0.95,0.091712
8,서울특별시,530721,1100000000,9384325,"4,471,017",2.10,"4,538,354","4,845,971",0.94,0.056554
9,세종특별자치시,27510,3600000000,386944,"160,942",2.40,"192,704","194,240",0.99,0.071096


In [ ]:
graph_viz(pd.DataFrame(sido_address_pop.groupby('시도명').sum()['인구 대비 도로명주소 개수']).sort_values(by='인구 대비 도로명주소 개수', ascending=False), '시도별 인구 대비 도로명주소 개수')

<ipython-input-35-d17e8f349037>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  graph_viz(pd.DataFrame(sido_address_pop.groupby('시도명').sum()['인구 대비 도로명주소 개수']).sort_values(by='인구 대비 도로명주소 개수', ascending=False), '시도별 인구 대비 도로명주소 개수')


#### 시군구별

In [ ]:
# 시군구별 도로명주소 개수 데이터
sigungu = pd.DataFrame(df.groupby(['시도명','시군구명'])["도로명주소관리번호"].count()).reset_index() # 데이터프레임 생성하기
sigungu.loc[:, "전체 시군구명"] = sigungu["시도명"] + " " + sigungu["시군구명"]
sigungu.drop(['시도명',"시군구명"], axis=1, inplace=True)
sigungu = sigungu[['전체 시군구명', '도로명주소관리번호']].copy()
sigungu['전체 시군구명'] = sigungu['전체 시군구명'].apply(lambda x:x.strip()) # 좌우 공백 없애기
sigungu['도로명주소관리번호'] = sigungu['도로명주소관리번호'].astype(int) # int로 데이터 타입 바꾸기

# 시군구별 인구수 데이터
sigungu_pop = df_pop[df_pop['행정기관'].isin(list(sigungu['전체 시군구명']))]  # 데이터프레임 생성하기
sigungu_pop['행정기관'] = sigungu_pop['행정기관'].apply(lambda x:x.strip()) # 좌우 공백 없애기
sigungu_pop['행정기관코드'] = sigungu_pop['행정기관코드'].astype('str') # str로 데이터 타입 바꾸기
sigungu_pop['총인구수'] = sigungu_pop['총인구수'].str.replace(',', '').astype(int) # int로 데이터 타입 바꾸기

<ipython-input-38-ae801cc19454>:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-38-ae801cc19454>:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-38-ae801cc19454>:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
# 하나의 데이터프레임으로 합치기
sigungu_address_pop = pd.merge(sigungu, sigungu_pop, left_on='전체 시군구명', right_on='행정기관')
sigungu_address_pop.drop('행정기관', axis=1,  inplace=True)
sigungu_address_pop.rename(columns={'도로명주소관리번호':'도로명개수'}, inplace=True)

# 인구 대비 도로명주소 개수 계산하기
sigungu_address_pop["인구 대비 도로명주소 개수"] = sigungu_address_pop.apply(lambda row: row["도로명개수"] / row["총인구수"], axis=1)
sigungu_address_pop

,전체 시군구명,도로명개수,행정기관코드,총인구수,세대수,세대당 인구,남자 인구수,여자 인구수,남여 비율,인구 대비 도로명주소 개수
0,강원특별자치도 강릉시,43680,5115000000,209174,"103,639",2.02,"103,616","105,558",0.98,0.208821
1,강원특별자치도 고성군,12749,5182000000,27294,"15,435",1.77,"14,008","13,286",1.05,0.467099
2,강원특별자치도 동해시,15172,5117000000,88591,"43,433",2.04,"45,007","43,584",1.03,0.171259
3,강원특별자치도 삼척시,19020,5123000000,62309,"33,500",1.86,"31,645","30,664",1.03,0.305253
4,강원특별자치도 속초시,12609,5121000000,82021,"41,353",1.98,"40,606","41,415",0.98,0.153729
...,...,...,...,...,...,...,...,...,...,...
246,충청북도 청주시 상당구,25361,4311100000,200726,"89,880",2.23,"99,728","100,998",0.99,0.126346
247,충청북도 청주시 서원구,21446,4311200000,186023,"88,696",2.10,"92,576","93,447",0.99,0.115287
248,충청북도 청주시 청원구,25894,4311400000,190559,"90,171",2.11,"98,111","92,448",1.06,0.135884
249,충청북도 청주시 흥덕구,27476,4311300000,274643,"130,318",2.11,"139,904","134,739",1.04,0.100043


In [ ]:
graph_viz(pd.DataFrame(sigungu_address_pop.groupby('전체 시군구명').sum()['인구 대비 도로명주소 개수']).sort_values(by='인구 대비 도로명주소 개수', ascending=False).head(20), '시군구별 인구 대비 도로명주소 개수')

<ipython-input-40-7eb80e089233>:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



## 분석2. 면적 대비 도로명주소 개수

### 면적 계산하기

In [ ]:
def cal_area(df1):
  df1['code'] = df1['code'].astype('str') # str로 데이터 타입 바꾸기

  # 면적 계산하기
  df1 = df1.set_crs(epsg=5179, allow_override=True)
  df1["면적"] = df1["geometry"].area
  return df1

# 시도별
sido_geojson = cal_area(sido_geojson)
# 시군구별
sigungu_geojson = cal_area(sigungu_geojson)

### 면적 대비 도로명주소 개수 계산하기

In [ ]:
def cal_area_per_road(df1, df2):
  df_total = df1.merge(df2, left_on='행정기관코드', right_on='code', how='outer')
  df_total["면적 대비 도로명주소 개수"] = df_total.apply(lambda row: row["도로명개수"] / row["면적"], axis=1)
  return df_total

# 시도별
sido_total = cal_area_per_road(sido_address_pop, sido_geojson)
# 시군구별
sigungu_total = cal_area_per_road(sigungu_address_pop, sigungu_geojson)

In [ ]:
graph_viz(pd.DataFrame(sido_total.groupby('시도명').sum()['면적 대비 도로명주소 개수']).sort_values(by='면적 대비 도로명주소 개수', ascending=False), '시도별 면적 대비 도로명주소 개수')

<ipython-input-46-565fef0bf126>:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [ ]:
# 누락된 세종시 데이터 추가하기
sigungu_total.loc[251] = sido_total[sido_total['시도명'] == '세종특별자치시'].iloc[0]
sigungu_total['전체 시군구명'][251] = '세종특별자치시'
sigungu_total

<ipython-input-47-708a77c96401>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,전체 시군구명,도로명개수,행정기관코드,총인구수,세대수,세대당 인구,남자 인구수,여자 인구수,남여 비율,인구 대비 도로명주소 개수,code,eng_nm,kor_nm,geometry,면적,면적 대비 도로명주소 개수
0,강원특별자치도 강릉시,43680.0,5115000000,209174.0,"103,639",2.02,"103,616","105,558",0.98,0.208821,5115000000,Gangneung-si,강릉시,"MULTIPOLYGON (((1136545.803 1962251.161, 11365...",1.038298e+09,0.000042
1,강원특별자치도 고성군,12749.0,5182000000,27294.0,"15,435",1.77,"14,008","13,286",1.05,0.467099,5182000000,Goseong-gun,고성군,"MULTIPOLYGON (((1091705.056 2034023.203, 10917...",5.855961e+08,0.000022
2,강원특별자치도 동해시,15172.0,5117000000,88591.0,"43,433",2.04,"45,007","43,584",1.03,0.171259,5117000000,Donghae-si,동해시,"MULTIPOLYGON (((1140184.975 1956251.627, 11401...",1.805981e+08,0.000084
3,강원특별자치도 삼척시,19020.0,5123000000,62309.0,"33,500",1.86,"31,645","30,664",1.03,0.305253,5123000000,Samcheok-si,삼척시,"MULTIPOLYGON (((1158822.812 1924922.029, 11588...",1.185369e+09,0.000016
4,강원특별자치도 속초시,12609.0,5121000000,82021.0,"41,353",1.98,"40,606","41,415",0.98,0.153729,5121000000,Sokcho-si,속초시,"MULTIPOLYGON (((1097565.776 2022623.133, 10975...",1.055047e+08,0.000120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,충청북도 청주시 서원구,21446.0,4311200000,186023.0,"88,696",2.10,"92,576","93,447",0.99,0.115287,4311200000,"Seowon-gu, Cheongju-si",청주시 서원구,"POLYGON ((998647.622 1842827.247, 998646.887 1...",1.231583e+08,0.000174
248,충청북도 청주시 청원구,25894.0,4311400000,190559.0,"90,171",2.11,"98,111","92,448",1.06,0.135884,4311400000,"Cheongwon-gu, Cheongju-si",청주시 청원구,"POLYGON ((1006036.714 1863981.585, 1006033.083...",2.146475e+08,0.000121
249,충청북도 청주시 흥덕구,27476.0,4311300000,274643.0,"130,318",2.11,"139,904","134,739",1.04,0.100043,4311300000,"Heungdeok-gu, Cheongju-si",청주시 흥덕구,"POLYGON ((983798.653 1858492.764, 983804.021 1...",1.987383e+08,0.000138
250,충청북도 충주시,44845.0,4313000000,207761.0,"102,152",2.03,"105,446","102,315",1.03,0.215849,4313000000,Chungju-si,충주시,"POLYGON ((1021661.932 1912518.233, 1021659.388...",9.808182e+08,0.000046


In [ ]:
graph_viz(pd.DataFrame(sigungu_total.groupby('전체 시군구명').sum()['면적 대비 도로명주소 개수']).sort_values(by='면적 대비 도로명주소 개수', ascending=False).head(20), '시군구별 면적 대비 도로명주소 개수')

<ipython-input-29-42191c3ab3b3>:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



## 최종 지도 시각화

In [ ]:
token = "pk.eyJ1IjoianlsZWVlIiwiYSI6ImNsMGtnM3lxbDBtMW0zY3Vvc2hzN3dibG8ifQ.PQMSK1vvdKW2DnTNIFtv4w"

### 시도별

In [ ]:
gdf_sido = gpd.GeoDataFrame(sido_total)

# mapbox에 맞춰 좌표계 변환
gdf_sido = gdf_sido.to_crs(epsg=4326)

gdf_sido.to_csv('3-5_sido-viz.csv', encoding='utf-8')
gdf_sido.to_file('sido-geoj.geojson', driver="GeoJSON")

gdf_sido.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 18 entries, 0 to 17
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   시도명             18 non-null     object  
 1   도로명개수           18 non-null     int64   
 2   행정기관코드          18 non-null     object  
 3   총인구수            18 non-null     int64   
 4   세대수             18 non-null     object  
 5   세대당 인구          18 non-null     object  
 6   남자 인구수          18 non-null     object  
 7   여자 인구수          18 non-null     object  
 8   남여 비율           18 non-null     object  
 9   인구 대비 도로명주소 개수  18 non-null     float64 
 10  code            17 non-null     object  
 11  eng_nm          17 non-null     object  
 12  kor_nm          17 non-null     object  
 13  geometry        17 non-null     geometry
 14  면적              17 non-null     float64 
 15  면적 대비 도로명주소 개수  17 non-null     float64 
dtypes: float64(3), geometry(1), int64(2), object(10)
memory 

In [ ]:
gdf_sido = pd.read_csv('3-5_sido-viz.csv', encoding='utf-8')
geo_data_sido = 'sido-geoj.geojson'

with open(geo_data_sido, 'rt', encoding='utf-8') as f_sido:
    gj_sido = geojson.load(f_sido)

In [ ]:
print(gdf_sido['인구 대비 도로명주소 개수'].describe())
print(gdf_sido['면적 대비 도로명주소 개수'].describe())

count    18.000000
mean      0.153094
std       0.090731
min       0.056554
25%       0.075910
50%       0.095722
75%       0.230680
max       0.331895
Name: 인구 대비 도로명주소 개수, dtype: float64
count    17.000000
mean      0.000160
std       0.000208
min       0.000022
25%       0.000054
50%       0.000084
75%       0.000175
max       0.000877
Name: 면적 대비 도로명주소 개수, dtype: float64


In [ ]:
viz = ChoroplethViz(data=gj_sido,
                    color_property='인구 대비 도로명주소 개수',
                    access_token=token,
                    color_stops=create_color_stops([0, 0.05, 0.1, 0.15, 0.2, 0.4], colors='BuPu'),
                    color_function_type='interpolate',
                    line_stroke='--',
                    line_color='rgb(128,0,38)',
                    line_width=1,
                    line_opacity=0.9,
                    opacity=0.8,
                    center = (128, 36),
                    zoom=5.5,
                    below_layer='waterway-label',
                    legend_layout='horizontal',
                    legend_key_shape='bar',
                    legend_key_borders_on=False)

html = open('person_per_address.html', "w", encoding="UTF-8")
html.write(viz.create_html())
html.close()

viz.show()

In [ ]:
# 맵을 -15도 만큼 좌우 회전하고, 45도 만큼 상하 회전합니다.
viz.bearing = -15
viz.pitch = 45

# 각 데이터에 '인구 대비 도로명주소 개수'를 기준으로 height 값을 줍니다.
viz.height_property = '인구 대비 도로명주소 개수'

## 높이의 값
numeric_stops = create_numeric_stops([0, 0.05, 0.1, 0.15, 0.2, 0.4], 0, 10000)

viz.height_stops = numeric_stops
viz.height_function_type = 'interpolate'

html = open('person_per_address_3d.html', "w", encoding="UTF-8")
html.write(viz.create_html())
html.close()

viz.show()

In [ ]:
viz = ChoroplethViz(data=gj_sido,
                    color_property='면적 대비 도로명주소 개수',
                    access_token=token,
                    color_stops=create_color_stops([0, 0.00002, 0.00005, 0.0001, 0.0002, 0.0009], colors='YlGnBu'),
                    color_function_type='interpolate',
                    line_stroke='--',
                    line_color='rgb(128,0,38)',
                    line_width=1,
                    line_opacity=0.9,
                    opacity=0.8,
                    center = (128, 36),
                    zoom=5.5,
                    below_layer='waterway-label',
                    legend_layout='horizontal',
                    legend_key_shape='bar',
                    legend_key_borders_on=False)

html = open('area_per_address.html', "w", encoding="UTF-8")
html.write(viz.create_html())
html.close()

viz.show()

### 시군구별

In [ ]:
gdf_sigungu = gpd.GeoDataFrame(sigungu_total)

# 좌표계 변환
gdf_sigungu = gdf_sigungu.to_crs(epsg=4326)

gdf_sigungu.to_csv('sigungu-viz.csv', encoding='utf-8')
gdf_sigungu.to_file('sigungu-geoj.geojson', driver="GeoJSON")

gdf_sigungu.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 252 entries, 0 to 251
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   전체 시군구명         252 non-null    object  
 1   도로명개수           252 non-null    float64 
 2   행정기관코드          252 non-null    object  
 3   총인구수            252 non-null    float64 
 4   세대수             252 non-null    object  
 5   세대당 인구          252 non-null    object  
 6   남자 인구수          252 non-null    object  
 7   여자 인구수          252 non-null    object  
 8   남여 비율           252 non-null    object  
 9   인구 대비 도로명주소 개수  252 non-null    float64 
 10  code            252 non-null    object  
 11  eng_nm          252 non-null    object  
 12  kor_nm          252 non-null    object  
 13  geometry        252 non-null    geometry
 14  면적              252 non-null    float64 
 15  면적 대비 도로명주소 개수  252 non-null    float64 
dtypes: float64(5), geometry(1), object(10)
memory usage: 4

In [ ]:
gdf_sigungu = pd.read_csv('sigungu-viz.csv', encoding='utf-8')
geo_data_sigungu = 'sigungu-geoj.geojson'

with open(geo_data_sigungu, 'rt', encoding='utf-8') as f_sigungu:
    gj_sigungu = geojson.load(f_sigungu)

In [ ]:
print(gdf_sigungu['인구 대비 도로명주소 개수'].describe())
print(gdf_sigungu['면적 대비 도로명주소 개수'].describe())

count    252.000000
mean       0.250847
std        0.212270
min        0.016889
25%        0.071034
50%        0.156822
75%        0.432216
max        0.775355
Name: 인구 대비 도로명주소 개수, dtype: float64
count    252.000000
mean       0.000291
std        0.000431
min        0.000007
25%        0.000044
50%        0.000086
75%        0.000324
max        0.002693
Name: 면적 대비 도로명주소 개수, dtype: float64


In [ ]:
viz = ChoroplethViz(data=gj_sigungu,
                    color_property='인구 대비 도로명주소 개수',
                    access_token=token,
                    color_stops=create_color_stops([0, 0.1, 0.2, 0.3, 0.4, 0.8], colors='BuPu'),
                    color_function_type='interpolate',
                    line_stroke='--',
                    line_color='rgb(128,0,38)',
                    line_width=1,
                    line_opacity=0.9,
                    opacity=0.8,
                    center = (128, 36),
                    zoom=5.5,
                    below_layer='waterway-label',
                    legend_layout='horizontal',
                    legend_key_shape='bar',
                    legend_key_borders_on=False)

html = open('sigungu_person_per_address.html', "w", encoding="UTF-8")
html.write(viz.create_html())
html.close()

viz.show()

/usr/local/lib/python3.10/dist-packages/IPython/core/display.py:724: UserWarning:

Consider using IPython.display.IFrame instead



In [ ]:
viz = ChoroplethViz(data=gj_sigungu,
                    color_property='면적 대비 도로명주소 개수',
                    access_token=token,
                    color_stops=create_color_stops([0, 0.0001, 0.00015, 0.0002, 0.00025, 0.003], colors='YlGnBu'),
                    color_function_type='interpolate',
                    line_stroke='--',
                    line_color='rgb(128,0,38)',
                    line_width=1,
                    line_opacity=0.9,
                    opacity=0.8,
                    center = (128, 36),
                    zoom=5.5,
                    below_layer='waterway-label',
                    legend_layout='horizontal',
                    legend_key_shape='bar',
                    legend_key_borders_on=False)

html = open('sigungu_area_per_address.html', "w", encoding="UTF-8")
html.write(viz.create_html())
html.close()

viz.show()